In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
import pandas_profiling

In [2]:
df=pd.read_csv("dataset.csv")

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df.drop_duplicates(keep='first',inplace=True)

In [6]:
df['Category'].value_counts()

ham     4516
spam     641
Name: Category, dtype: int64

In [7]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [42]:
def process_text(text):

    nopunc=[char for char in text if char not in string.punctuation]
    nopunc=''.join(nopunc)
    
    clean_words=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [43]:
df['Message'].head().apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [45]:
msg4='hello world hello hello world play'
msg5='test test test test one hello'

print(msg4)
print()

from sklearn.feature_extraction.text import CountVectorizer
bow4=CountVectorizer(analyzer=process_text).fit_transform([[msg4],[msg5]])
print(bow4)
print()

hello world hello hello world play

  (0, 0)	3
  (0, 4)	2
  (0, 2)	1
  (1, 0)	1
  (1, 3)	4
  (1, 1)	1



In [47]:
message_bow=CountVectorizer(analyzer=process_text).fit_transform(df['Message'])

In [52]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(message_bow,df['Category'],test_size=0.2,random_state=42)

In [53]:
message_bow.shape

(5157, 11422)

In [107]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB(alpha=3.5)
classifier.fit(X_train,y_train)

MultinomialNB(alpha=3.5, class_prior=None, fit_prior=True)

In [108]:
print(classifier.predict(X_train))

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [109]:
print(y_train.values)

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [110]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [111]:
pred=classifier.predict(X_train)

In [112]:
print(classification_report(y_train,pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      3620
        spam       1.00      0.90      0.94       505

    accuracy                           0.99      4125
   macro avg       0.99      0.95      0.97      4125
weighted avg       0.99      0.99      0.99      4125



In [113]:
confusion_matrix(y_train,pred)

array([[3618,    2],
       [  52,  453]], dtype=int64)

In [114]:
accuracy_score(y_train,pred)*100

98.6909090909091

In [115]:
y_pred=classifier.predict(X_test)

In [116]:
confusion_matrix(y_test,y_pred)

array([[886,  10],
       [ 14, 122]], dtype=int64)

In [117]:
accuracy_score(y_test,y_pred)*100

97.67441860465115

In [118]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n         ham       0.98      0.99      0.99       896\n        spam       0.92      0.90      0.91       136\n\n    accuracy                           0.98      1032\n   macro avg       0.95      0.94      0.95      1032\nweighted avg       0.98      0.98      0.98      1032\n'